# Serverless Deployment of a machine learning model in AWS

## Machine learning model

In [20]:
# Loading necessary libraries
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, accuracy_score, recall_score, plot_confusion_matrix, classification_report, confusion_matrix
import pandas as pd
import numpy as np
import pickle

In [22]:
# Loading dataset
X, Y = load_breast_cancer(return_X_y=True)

# creating train and test dataset
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

In [24]:
# Optimum model search for the classification
n_estimators = [50, 100, 150, 200]
max_depth = [5, 10, 25, 50, 100, 200]

for i in range(len(n_estimators)):
    for j in range(len(max_depth)):
        classifier = RandomForestClassifier(n_estimators=n_estimators[i], max_depth=max_depth[j])
        classifier.fit(x_train, y_train)
        y_pred = classifier.predict(x_train)
        f1_score_t = f1_score(y_train, y_pred, average='micro')
        y_pred_test = classifier.predict(x_test)
        f1_score_test = f1_score(y_test, y_pred_test, average='micro')
        print_out = 'Random Forest Classifier number of estimators ' +  str(n_estimators[i]) +  ' max depth ' + str(max_depth[j]) + ' training dataset F1 Score is ' + str(f1_score_t) + ' validation dataset F1 Score is ' + str(f1_score_test)
        print(print_out)


Random Forest Classifier number of estimators 50 max depth 5 training dataset F1 Score is 0.9934065934065934 validation dataset F1 Score is 0.9824561403508771
Random Forest Classifier number of estimators 50 max depth 10 training dataset F1 Score is 1.0 validation dataset F1 Score is 0.9736842105263158
Random Forest Classifier number of estimators 50 max depth 25 training dataset F1 Score is 1.0 validation dataset F1 Score is 0.9736842105263158
Random Forest Classifier number of estimators 50 max depth 50 training dataset F1 Score is 1.0 validation dataset F1 Score is 0.9736842105263158
Random Forest Classifier number of estimators 50 max depth 100 training dataset F1 Score is 1.0 validation dataset F1 Score is 0.9824561403508771
Random Forest Classifier number of estimators 50 max depth 200 training dataset F1 Score is 1.0 validation dataset F1 Score is 0.9824561403508771
Random Forest Classifier number of estimators 100 max depth 5 training dataset F1 Score is 0.9912087912087912 vali

In [27]:
# Best model for the classification
classifier = RandomForestClassifier(n_estimators=50, max_depth=5)
classifier.fit(x_train, y_train)

y_pred = classifier.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96        39
           1       0.99      0.97      0.98        75

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114



In [28]:
# exporting the model with pickle
pickle.dump( classifier, open( "classification_model.p", "wb" ) )

## Deployment of the model to AWS

The model

- Copy the pickled model to S3 in AWS through command line in the same directory as your notebook run : "aws s3 cp classification_model.p s3://kashifs3bucket" with the S3 bucket name at the end
- To deploy the model will use SAM “Serverless Application Model”. It’s an open-source framework for provisioning AWS services via code.
- Run "SAM init" in the comman line in the same directory as your notebook
    - Choosing an AWS quick start template
    - Package type as zip
    - Runtime as python3.8
    - Project name
    - and as template Hello World Example
    - Chnaged the name of the hell_world folder to code
- Update the template.yaml document for the required resources in AWS 
    - By creating the lambda function 
    - Letting know from where to download the model
    - Creating API gateway to connect to the Lambda function from externally
    - Need to create the role for lambda function to access the S3 resources
- Modify the app.py in the code folder to appropriately handle the lambdda function
